In [1]:
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local"))

import re

def parse_article(line):
    try:
        article_id, text = line.rstrip().split('\t', 1)
        text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
        words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
        return words
    except ValueError as e:
        return []
    
def pair_words(words):
    result = list()
    for word1, word2 in zip(words, words[1:]):
        result.append((word1.lower() + "_" + word2.lower(), 1))
    return result
    
result = (sc.textFile("/data/wiki/en_articles_part/articles-part", 16)
        .map(parse_article)
        .flatMap(pair_words)
        .reduceByKey(lambda x,y : x+y)
        .filter(lambda word: word[0][:9] == "narodnaya")
       ).collect()

for key, value in result:
    print("{0}\t{1}".format(key,value))

narodnaya_gazeta	1
narodnaya_volya	9
